# ImageNet

We created a supervised machine learning bdp for MNIST, but what if we want to re-use this for ImageNet? Let's quickly write out what would need to happen!

In [1]:
import sys
sys.path.append("../../pybdp")
from src import pybdp
#import pybdp
from pprint import pprint

# Start with an empty project
project = pybdp.load_from_json("../JSON/Supervised Learning V2.json")
print(project)

< Project
Toolbox:

< Toolbox
Blocks: ['Experiment', 'Load Supervised Features', 'Supervised Learning', 'Load Supervised Dataset', 'Cross Validation Split', 'Training Data Preprocessing', 'Testing Data Preprocessing', 'Fit Supervised Model', 'Evaluate Supervised Model', 'Post-processing']
Spaces: ['Model', 'Evaluation Metrics', 'X Train', 'Y Train', 'X Test', 'Y Test', 'X', 'Y'] >

Workbench:

<Workbench
Processors: ['MNIST Experiment', 'Load MNIST', 'Default Supervised Learning', 'Load MNIST Dataset', 'Test-Train Split', 'Image Normalization Preprocessing - Training', 'Image Normalization Preprocessing - Testing', 'Fit Supervised Model - Default', 'Evaluate Supervised Model - Default', 'No Post-processing']
Wires: ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 'W14']
Systems: ['MNIST Experiment System', 'Load MNIST System', 'Default Supervised Learning System'] > >


In [2]:
project.copy_add_processor(project.processors_map["Load MNIST Dataset"],
                           {"ID": "Load ImageNet Dataset",
                            "Name": "Load ImageNet Dataset",
                            "Description": "Loads the ImageNet dataset"},
                            copy_wires=True)
print(project)

< Project
Toolbox:

< Toolbox
Blocks: ['Experiment', 'Load Supervised Features', 'Supervised Learning', 'Load Supervised Dataset', 'Cross Validation Split', 'Training Data Preprocessing', 'Testing Data Preprocessing', 'Fit Supervised Model', 'Evaluate Supervised Model', 'Post-processing']
Spaces: ['Model', 'Evaluation Metrics', 'X Train', 'Y Train', 'X Test', 'Y Test', 'X', 'Y'] >

Workbench:

<Workbench
Processors: ['MNIST Experiment', 'Load MNIST', 'Default Supervised Learning', 'Load MNIST Dataset', 'Test-Train Split', 'Image Normalization Preprocessing - Training', 'Image Normalization Preprocessing - Testing', 'Fit Supervised Model - Default', 'Evaluate Supervised Model - Default', 'No Post-processing', 'Load ImageNet Dataset']
Wires: ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 'W14', 'W15', 'W16']
Systems: ['MNIST Experiment System', 'Load MNIST System', 'Default Supervised Learning System'] > >


In [ ]:
add_wires = [x.id for x in project.processors_map["Load ImageNet Dataset"].wires]
remove_wires = [x.id for x in project.processors_map["Load MNIST Dataset"].wires]

add_dictionary = {"Wires": add_wires,
                  "Processors": ["Load ImageNet Dataset"]}
remove_dictionary = {"Wires": remove_wires,
                     "Processors": ["Load MNIST Dataset"]}
update_dictionary = {"ID": "Load ImageNet System",
                     "Name": "Load ImageNet System",
                    "Description": "The system representing loading the ImageNet dataset, preparing the cross validation split, and pre-processing the data"}

project.copy_add_system(project.systems_map["Load MNIST System"],
                        update_dictionary,
                        add_dictionary=add_dictionary,
                        remove_dictionary=remove_dictionary,)

['W15', 'W16'] ['W5', 'W6']
